# Gemini models in  SAP AI Core

Using direct API calls with Vertex AI format.


## 1. Setup


In [2]:
import os
import json
import requests


## 2. Set Credentials


In [3]:
# Path to your AI Core service key JSON file
service_key_path = "../aicore-service-key.json"

# Load service key
with open(service_key_path, 'r') as f:
    service_key = json.load(f)

# Set environment variables
os.environ['AICORE_AUTH_URL'] = service_key['url']
os.environ['AICORE_CLIENT_ID'] = service_key['clientid']
os.environ['AICORE_CLIENT_SECRET'] = service_key['clientsecret']
os.environ['AICORE_RESOURCE_GROUP'] = 'default'

# Parse serviceurls - it might be a string or already a dict
serviceurls = service_key['serviceurls']
if isinstance(serviceurls, str):
    serviceurls = json.loads(serviceurls)
os.environ['AICORE_BASE_URL'] = serviceurls['AI_API_URL']
base_url = os.environ['AICORE_BASE_URL']

print("✓ AI Core credentials configured")


✓ AI Core credentials configured


## 3. Get Auth Token


In [4]:
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

# Get auth headers
proxy = get_proxy_client('gen-ai-hub')
headers = proxy.get_request_header()

print('✓ Auth token obtained')


✓ Auth token obtained


## Check for Image Generation Models

In [5]:
# List all deployments and check for image generation models
deployments = proxy.get_deployments()

print('All Available Deployments:')
print('='*80)

image_models = []

for d in deployments:
    model_name = d.model_name.lower()
    if "gemini" in model_name:
        print(f"\nModel: {d.model_name}")
        print(f"  Deployment ID: {d.deployment_id}")
        
        # Check if it's an image generation model
        if any(x in model_name for x in ['dall-e', 'dalle', 'stable-diffusion', 'sd-', 'imagen']):
            image_models.append(d)
            print("  ✓ IMAGE GENERATION MODEL")
        else:
            
            print("  📝 Text/Analysis model")

    print('\n' + '='*80)
    print(f'\nSummary: {len(image_models)} image generation model(s) found')

    if image_models:
        print('\n✓ Image generation models:')
        for m in image_models:
            print(f'  - {m.model_name} (ID: {m.deployment_id})')
    else:
        print('\n⚠️  No image generation models deployed in your SAP AI Core')

All Available Deployments:


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core

Model: gemini-2.5-pro
  Deployment ID: d6ae523ed14c6cc3
  📝 Text/Analysis model


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core

Model: gemini-2.0-flash
  Deployment ID: d83ad32cbda0399d
  📝 Text/Analysis model


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generation models deployed in your SAP AI Core


Summary: 0 image generation model(s) found

⚠️  No image generati

## Call Gemini 2.5 Pro (Vertex AI Format)

In [6]:
# Gemini 2.5 Pro deployment
DEPLOYMENT_ID = 'd6ae523ed14c6cc3'
url = f'{base_url}/v2/inference/deployments/{DEPLOYMENT_ID}/models/gemini-2.5-pro:generateContent'

# Vertex AI format payload
payload = {
    'contents': [
        {
            'role': 'user',
            'parts': [{'text': 'can you generate the cat image'}]
        }
    ],
    'generationConfig': {
        'temperature': 0.7,
        'maxOutputTokens': 10000
    }
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print('✓ Success!')
    print(result)
    print(f"\nResponse:\n{result['candidates'][0]['content']['parts'][0]['text']}")
else:
    print(f'Error {response.status_code}: {response.text}')

✓ Success!
{'candidates': [{'avgLogprobs': -3.7101336980270125, 'content': {'parts': [{'text': 'Of course! Here is an image of a cat for you.\n\n'}, {'text': '\n\nWould you like to see a different kind of cat? For example, a specific breed (like a Siamese or a Maine Coon), a kitten, or a cat in a different style (like a cartoon or a painting)?'}], 'role': 'model'}, 'finishReason': 'STOP'}], 'createTime': '2025-11-20T08:55:36.590586Z', 'modelVersion': 'gemini-2.5-pro', 'responseId': 'iNceafqFJJ_BgLUP4tqViQU', 'usageMetadata': {'candidatesTokenCount': 59, 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 59}], 'promptTokenCount': 6, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 6}], 'thoughtsTokenCount': 1226, 'totalTokenCount': 1291, 'trafficType': 'ON_DEMAND'}}

Response:
Of course! Here is an image of a cat for you.


